In [1]:
import shutil
from glob import glob

import tensorboard_reducer as tbr
import os
from pathlib import Path


In [7]:
p = Path('runs/test') / 'sd'

In [8]:
p.parent.name

'test'

In [3]:
# where are your TensorBoard event files stored?
input_event_dirs = sorted(glob("runs/test/*"))

joined_dirs = "\n".join(input_event_dirs)
print(f"Found {len(input_event_dirs)} runs:\n{joined_dirs}")

Found 4 runs:
runs/test/Walker2d-v4__ddpg_continuous_action__37__1714450121
runs/test/Walker2d-v4__dgum_continuous_action__37__1714450021
runs/test/Walker2d-v4__sac_continuous_action__37__1714450389
runs/test/Walker2d-v4__td3_continuous_action__37__1714450225


In [5]:
events_out_dir = "tmp/reduced"
csv_out_path = "tmp/tb-reduction.csv"
overwrite = True
reduce_ops = ("mean", "min", "max", "std", "median")

events_dict = tbr.load_tb_events(input_event_dirs, verbose=True, strict_tags=False, strict_steps=False)

Reading tags: 100%|██████████| 4/4 [00:00<00:00, 107.12it/s]

Loaded data for 13 tags into arrays of shape (n_steps, n_runs):
- 'eval/training_avg': (20, 4)
- 'charts/episodic_return': (469, 4)
- 'charts/episodic_length': (469, 4)
- 'losses/qf1_values': (99, 4)
- 'losses/qf1_loss': (99, 4)
- 'losses/actor_loss': (99, 4)
- 'charts/SPS': (99, 4)
- 'eval/episodic_return': (10, 4)
- 'losses/qf2_values': (99, 2)
- 'losses/qf2_loss': (99, 2)
- 'losses/qf_loss': (99, 2)
- 'losses/alpha': (99, 1)
- 'losses/alpha_loss': (99, 1)


In [6]:
reduced_events = tbr.reduce_events(events_dict, reduce_ops, verbose=True)

for op in reduce_ops:
    print(f"Writing '{op}' reduction to '{events_out_dir}-{op}'")

tbr.write_tb_events(reduced_events, events_out_dir, overwrite, verbose=True)

print(f"Writing results to '{csv_out_path}'")

tbr.write_data_file(reduced_events, csv_out_path, overwrite, verbose=True)

print("✓ Reduction complete")

Reduced 13 scalars with 3 operations: (mean, min, max)
Writing 'mean' reduction to 'tmp/reduced-mean'
Writing 'min' reduction to 'tmp/reduced-min'
Writing 'max' reduction to 'tmp/reduced-max'


/home/rswamy/.conda/envs/cleanrl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Writing max reduction to disk: 100%|██████████| 3/3 [00:00<00:00,  3.94it/s] 

Created new TensorBoard event files in
- tmp/reduced-mean
- tmp/reduced-min
- tmp/reduced-max
Writing results to 'tmp/tb-reduction.csv'
Created new data file at 'tmp/tb-reduction.csv'
✓ Reduction complete
